# Lecture 1: Introduction to Functional Programming

This is a lecture from [Functional Programming Course](https://soshnikov.com/courses/funcpro/) by [Dmitry Soshnikov](http://soshnikov.com).

Let's start exploring F# programming language.

### Application and Abstraction

Two basic notions of lambda-calculus are application and abstraction:

In [4]:
sqrt 4.0

2

In [6]:
(fun x->x+1)

In [8]:
(fun x->x+1) 12

13

In [12]:
(fun f x -> f (f x)) (fun x->x*2) 1

4

### Let let's you use intermediate names

In [16]:
let rpt2 = fun f -> fun x -> f (f x)
let twice = fun x -> x*2
rpt2 twice 1

4

In [17]:
let rpt2 f x = f (f x)
let twice x = x*2
rpt2 twice 1

4

### Composition!

In [18]:
twice (twice 2)

8

In [19]:
2 |> twice |> twice

8

In [20]:
(twice >> twice) 2

8

In [21]:
twice <| 2 |> twice

8

### Functions of Many Parameters: Currying

In [22]:
let plus (a,b) = a+b

plus (1,2)

3

In [23]:
let plus a b = a+b

plus 1 2 

3

In [24]:
(plus 1) 2

3

In [25]:
let p1 = plus 1
p1 2

3

In [27]:
let p1 = (+) 1
p1 12

13

### Problem

Define the following functions. Then, try to define them using ONLY function composition and currying, without explicit lambdas:
 * $f(x) = 2x$
 * $f(x) = 2x+1$
 * $f(x) = 2(x+1)$

### Example: Solving Quadratic Equation

In [29]:
let solve a b c = 
  let D = b*b-4.*a*c
  let x1 = (-b+sqrt(D))/2./a
  let x2 = (-b-sqrt(D))/2./a
  (x1,x2)
  
solve 1. 2. -3.

Item1,Item2
1,-3


In [30]:
solve 1. 2. 3.

Item1,Item2
NaN,NaN


In [34]:
let solve a b c = 
  let D = b*b-4.*a*c
  if D<0. then (999.9,999.9)
  else
      let x1 = (-b+sqrt(D))/2./a
      let x2 = (-b-sqrt(D))/2./a
      (x1,x2)
  
solve 1. 2. 3.

Item1,Item2
999.9,999.9


In [37]:
type QResult =
   | Quad of float*float
   | Linear of float
   | NoRoots
   
let solve a b c =
  if a=0. then Linear(-c/b)
  else 
      let D = b*b-4.*a*c
      if D<0. then NoRoots
      else
          let x1 = (-b+sqrt(D))/2./a
          let x2 = (-b-sqrt(D))/2./a
          Quad(x1,x2)
  
solve 1. 2. 3.

NoRoots

In [40]:
let print res = 
  match res with
   | NoRoots -> printfn "No Roots"
   | Quad(x1,x2) when x1=x2 -> printfn "x1=x2=%f" x1
   | Quad(x1,x2) -> printfn "x1=%f, x2=%f" x1 x2
   | Linear(x) -> printfn "x=%f" x

solve 1. 2. 3. |> print

No Roots


In [44]:
let print = function
   | NoRoots -> printfn "No Roots"
   | Quad(x1,x2) when x1=x2 -> printfn "x1=x2=%f" x1
   | Quad(x1,x2) -> printfn "x1=%f, x2=%f" x1 x2
   | Linear(x) -> printfn "x=%f" x

solve 1. 2. 1. |> print

x1=x2=-1.000000


### TAYLOR Series

Let's define function to calculate $e^x$ using Taylor series, eg.:
$$
e^x = 1+x+{x^2\over 2!}+\dots+{x^n\over n!}+\dots
$$

In [ ]:
let rec fact n = function
| 1 -> 1
| n -> n*fact(n-1)

let pow x = function
| 0 -> 1
| n -> x*pow x (n-1)

let myexp' x = function
| 0 -> 1
| n -> myexp' x (n-1) + (pow x n/float(fact n))

let myexp x = myexp' x 0

myexp 1.

In [1]:
let rec iter f i a b =
    if a>b then i
    else f a (iter f i (a+1) b)
    
iter (fun x acc -> x+acc) 0 1 100

The below script needs to be able to find the current output cell; this is an easy method to get it.

5050

In [2]:
let sum = iter (+) 0
sum 1 100

5050

In [3]:
let fact = iter (*) 1 1
fact 5

120

In [4]:
let fact = iter (*) 1 1
let pow x = iter (fun _ acc -> x*acc) 1. 0
let myexp x = iter (fun n acc -> acc+(pow x n/float(fact n))) 1. 1 5

myexp 1.

2.716666666666667

### Mandelbrot Set

In [5]:
open System.Numerics

let mandel_seq c (z:Complex) = z*z+c

In [6]:
let rpt n f = iter (fun _ t -> f>>t) (fun x->x) 1 n

rpt 8 ((*)2) 1

256

In [7]:
let is_mandel c = Complex.Abs(rpt 20 (mandel_seq c) Complex.Zero)<1.0

In [8]:
for i = 0 to 20 do
  for j = 0 to 60 do
     let c = new Complex((float(j)-30.0)/20.0,(float(i)-10.0)/10.0)
     printf "%c" (if is_mandel c then '*' else ' ')
  printfn ""

                                                             
                           *                                 
                          ***                                
                          ****                               
                   ** ***********                            
                   ******************                        
                  *******************                        
          *     **********************                       
       *******  ***********************                      
      ********************************                       
************************************                         
      ********************************                       
       *******  ***********************                      
          *     **********************                       
                  *******************                        
                   ******************                        
        

In [ ]:
open System.Drawing
let image = new Bitmap(400, 400)
for i = 0 to (image.Height-1) do
    for j = 0 to (image.Width-1) do
       let c = new Complex((float(i)-200.0)/100.0,(float(j)-200.0)/100.0)
       image.SetPixel(i,j,if is_mandel c then Color.Black else Color.White)
image.Save("mandelbrot.jpg")
{ Html = @"<img src=""mandelbrot.jpg""/>"}